# Phase 3 QoS Analysis Scratchpad

This notebook runs the Phase 3 latency metrics query (scoped to the rolling 7-day window to control cost) and derives:

- Latency and slot usage statistics per consumer classification.
- Slot utilization percentiles relative to reservation capacity.
- Concurrency vs. the current reservation slot pool.

> **Guardrail:** the full query across all historical windows scans ~35 GB. The notebook limits execution to `rolling_07d` by default; adjust the helper parameters if you need broader windows and confirm expected scan cost with `--dry_run` first.


**Data sources:** SQL outputs are pre-generated and stored alongside this notebook. Specifically:
- `rolling_07d_latency.json` comes from running `new_audit_sql/phase3_qos_latency_metrics.sql` with `DECLARE window_ids = ['rolling_07d']` and saving the CLI JSON output.
- `rolling_07d_slot_usage_10min.json` comes from `new_audit_sql/phase3_qos_slot_usage_10min.sql` with the same window filter.
- `reservations_us.json` is a snapshot of `bq ls --reservation --project_id=bq-narvar-admin --location=US --format=prettyjson`.
These cached files let the notebook iterate quickly without reissuing the expensive BigQuery scans. Update them whenever you rerun the underlying SQL for a different window.


In [3]:
import json
import pathlib
import subprocess
from textwrap import dedent

import pandas as pd

ROOT = pathlib.Path('..').resolve().parent
SQL_DIR = ROOT / 'analysis_peak_2025_gpt_codex' / 'new_audit_sql'

print(f"Using SQL directory: {SQL_DIR}")


Using SQL directory: /Users/cezarmihaila/workspace/do_it_query_optimization_queries/bigquery-optimization-queries/narvar/analysis_peak_2025_gpt_codex/new_audit_sql


In [4]:
def run_bq_query(sql: str) -> pd.DataFrame:
    """Execute a SQL string with the bq CLI and return a DataFrame."""
    completed = subprocess.run(
        ["bq", "query", "--use_legacy_sql=false", "--format=prettyjson"],
        input=sql.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        check=False,
    )
    if completed.returncode != 0:
        raise RuntimeError(f"bq query failed: {completed.stderr.decode('utf-8')}")
    text = completed.stdout.decode("utf-8").strip()
    data = json.loads(text)
    return pd.json_normalize(data)


def load_sql(name: str) -> str:
    """Read a SQL file from the Phase 3 directory."""
    sql_path = SQL_DIR / name
    sql_text = sql_path.read_text()
    return sql_text


In [5]:
latency_sql = load_sql('phase3_qos_latency_metrics.sql').rstrip(';\n ')

start_token = "WITH qos_windows AS ("
end_token = "),\nbase_jobs AS ("
if start_token not in latency_sql or end_token not in latency_sql:
    raise ValueError("Unexpected SQL structure; cannot isolate qos_windows CTE")

prefix, remainder = latency_sql.split(start_token, 1)
windows_body, rest = remainder.split(end_token, 1)

filtered_windows_cte = """WITH qos_windows AS (
  SELECT 'rolling_07d' AS window_id,
         TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) AS start_ts,
         CURRENT_TIMESTAMP() AS end_ts
),
"""

latency_sql_filtered = prefix + filtered_windows_cte + rest

wrapped_latency_sql = f"""
WITH latency AS (
{latency_sql_filtered}
)
SELECT *
FROM latency;
"""

print("Prepared latency query (truncated):\n", wrapped_latency_sql[:500], "...", sep="")


Prepared latency query (truncated):

WITH latency AS (
-- Phase 3 QoS latency metrics per consumer classification and analysis window.
-- Computes queue time, run time, and total duration quantiles plus slot usage.

DECLARE window_ids ARRAY<STRING> DEFAULT [
  'peak_fy22', 'baseline_fy22',
  'peak_fy23', 'baseline_fy23',
  'peak_fy24', 'baseline_fy24',
  'baseline_fy25',
  'rolling_90d', 'rolling_28d', 'rolling_07d'
];

WITH qos_windows AS (
  SELECT 'rolling_07d' AS window_id,
         TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL ...


In [6]:
latency_results_path = pathlib.Path('rolling_07d_latency.json').resolve()
with open(latency_results_path) as f:
    latency_raw = json.load(f)
latency_df = pd.json_normalize(latency_raw[0])
latency_df


,avg_active_slots,avg_queue_seconds,avg_run_seconds,avg_total_seconds,classification_type,event_name,job_count,p50_queue_seconds,p50_run_seconds,p50_total_seconds,p90_queue_seconds,p90_run_seconds,p90_total_seconds,p99_queue_seconds,p99_run_seconds,p99_total_seconds,total_run_seconds,total_slot_ms,window_id
0,9.389795893719807,0.0,4.6516853932584254,4.773876404494383,AUTOMATION,extract_job_completed,712,0,5,5,0,6,6,0,8,9,3312,31099004,rolling_07d
1,7.886442157926168,0.0010002180926668225,4.936474870460478,5.140699851847396,AUTOMATION,load_job_completed,132971,0,2,2,0,12,12,0,28,28,656408,5176723724,rolling_07d
2,132.857917633584,0.00355080689000396,0.49604398023776214,0.50790608587208,AUTOMATION,query_job_completed,4115121,0,0,0,0,0,0,0,2,2,2041281,271200342965,rolling_07d
3,2.8619713878158164,4.7798291211089214E-4,32.2939594909482,32.4770866941507,HUB_SERVICE,load_job_completed,16737,0,31,31,0,46,46,0,56,56,540504,1546906983,rolling_07d
4,138.5390421469683,0.15272426254251212,12.881850550371158,13.167588107594204,HUB_SERVICE,query_job_completed,300797,0,0,0,0,3,3,0,54,57,3874822,536814128370,rolling_07d
5,0.1705,0.0,4.0,4.0,INTERNAL_USER,extract_job_completed,1,0,4,4,0,4,4,0,4,4,4,682,rolling_07d
6,0.8077759882869693,0.0,3.623342175066313,3.798408488063661,INTERNAL_USER,load_job_completed,377,0,2,2,0,6,6,0,12,12,1366,1103422,rolling_07d
7,401.2251889155564,0.0030736493675375346,41.27627379122826,41.4522993261615,INTERNAL_USER,query_job_completed,8459,0,0,0,0,24,24,0,96,97,349156,140090182061,rolling_07d
8,61.06397727272727,0.0,3.3846153846153846,3.4615384615384617,UNKNOWN,extract_job_completed,13,0,4,4,0,6,6,0,7,7,44,2686815,rolling_07d
9,59.3067924773022,0.0,8.862068965517242,8.999999999999998,UNKNOWN,load_job_completed,87,0,2,2,0,34,34,0,58,58,771,45725537,rolling_07d


### Load latency metrics
Read the rolling-7d latency query output (`rolling_07d_latency.json`) into a DataFrame and coerce numeric columns. This file is the raw response from `phase3_qos_latency_metrics.sql` filtered to the `rolling_07d` window.


In [7]:
numeric_cols = [
    'job_count',
    'total_slot_ms',
    'total_run_seconds',
    'avg_active_slots',
    'avg_queue_seconds',
    'avg_run_seconds',
    'avg_total_seconds',
    'p50_queue_seconds', 'p90_queue_seconds', 'p99_queue_seconds',
    'p50_run_seconds', 'p90_run_seconds', 'p99_run_seconds',
    'p50_total_seconds', 'p90_total_seconds', 'p99_total_seconds'
]
for col in numeric_cols:
    latency_df[col] = pd.to_numeric(latency_df[col], errors='coerce')
latency_df.head()


,avg_active_slots,avg_queue_seconds,avg_run_seconds,avg_total_seconds,classification_type,event_name,job_count,p50_queue_seconds,p50_run_seconds,p50_total_seconds,p90_queue_seconds,p90_run_seconds,p90_total_seconds,p99_queue_seconds,p99_run_seconds,p99_total_seconds,total_run_seconds,total_slot_ms,window_id
0,9.389796,0.000000,4.651685,4.773876,AUTOMATION,extract_job_completed,712,0,5,5,0,6,6,0,8,9,3312,31099004,rolling_07d
1,7.886442,0.001000,4.936475,5.140700,AUTOMATION,load_job_completed,132971,0,2,2,0,12,12,0,28,28,656408,5176723724,rolling_07d
2,132.857918,0.003551,0.496044,0.507906,AUTOMATION,query_job_completed,4115121,0,0,0,0,0,0,0,2,2,2041281,271200342965,rolling_07d
3,2.861971,0.000478,32.293959,32.477087,HUB_SERVICE,load_job_completed,16737,0,31,31,0,46,46,0,56,56,540504,1546906983,rolling_07d
4,138.539042,0.152724,12.881851,13.167588,HUB_SERVICE,query_job_completed,300797,0,0,0,0,3,3,0,54,57,3874822,536814128370,rolling_07d


In [8]:
slot_percentiles = (
    latency_df.groupby(['classification_type', 'event_name'])['total_slot_ms']
    .quantile([0.5, 0.9, 0.99])
    .unstack(level=-1)
    .rename(columns={0.5: 'p50', 0.9: 'p90', 0.99: 'p99'})
)
slot_percentiles


p50           p90  \
classification_type event_name                                             
AUTOMATION          extract_job_completed     3.109900e+07  3.109900e+07   
                    load_job_completed        5.176724e+09  5.176724e+09   
                    query_job_completed       2.712003e+11  2.712003e+11   
HUB_SERVICE         load_job_completed        1.546907e+09  1.546907e+09   
                    query_job_completed       5.368141e+11  5.368141e+11   
INTERNAL_USER       extract_job_completed     6.820000e+02  6.820000e+02   
                    load_job_completed        1.103422e+06  1.103422e+06   
                    query_job_completed       1.400902e+11  1.400902e+11   
UNKNOWN             extract_job_completed     2.686815e+06  2.686815e+06   
                    load_job_completed        4.572554e+07  4.572554e+07   
                    query_job_completed       3.514025e+10  3.514025e+10   
                    table_copy_job_completed  0.000000e+00  0.000000e+00   

                                                       p99  
classification_type event_name                              
AUTOMATION          extract_job_completed     3.109900e+07  
                    load_job_completed        5.176724e+09  
                    query_job_completed       2.712003e+11  
HUB_SERVICE         load_job_completed        1.546907e+09  
                    query_job_completed       5.368141e+11  
INTERNAL_USER       extract_job_completed     6.820000e+02  
                    load_job_completed        1.103422e+06  
                    query_job_completed       1.400902e+11  
UNKNOWN             extract_job_completed     2.686815e+06  
                    load_job_completed        4.572554e+07  
                    query_job_completed       3.514025e+10  
                    table_copy_job_completed  0.000000e+00

In [9]:
reservations_path = pathlib.Path('reservations_us.json').resolve()
with open(reservations_path) as f:
    reservations_raw = json.load(f)
reservations_df = pd.json_normalize(reservations_raw)
reservations_df


,creationTime,edition,ignoreIdleSlots,name,updateTime,autoscale.maxSlots,slotCapacity,autoscale.currentSlots
0,2024-06-28T17:24:59.668513Z,STANDARD,True,projects/bq-narvar-admin/locations/US/reservat...,2024-06-28T17:24:59.668513Z,300,NaN,NaN
1,2022-04-29T21:13:02.192290Z,ENTERPRISE,NaN,projects/bq-narvar-admin/locations/US/reservat...,2025-10-31T13:19:01.830464Z,700,1000,50


In [10]:
latency_summary = (
    latency_df.groupby('classification_type')
    .agg(
        job_count=('job_count', 'sum'),
        total_slot_ms=('total_slot_ms', 'sum'),
        total_run_seconds=('total_run_seconds', 'sum'),
        avg_active_slots=('avg_active_slots', 'mean')
    )
    .reset_index()
)
latency_summary


,classification_type,job_count,total_slot_ms,total_run_seconds,avg_active_slots
0,AUTOMATION,4248804,276408165693,2701001,50.044719
1,HUB_SERVICE,317534,538361035353,4415326,70.700507
2,INTERNAL_USER,8837,140091286165,350526,134.067822
3,UNKNOWN,3280,35188663270,147556,89.961729


### Reservation metadata
Load `reservations_us.json` (captured via `bq ls --reservation`) to pull committed slots and autoscale headroom for `projects/bq-narvar-admin/locations/US/reservations/default`. These values anchor the capacity comparisons.


In [11]:
default_row = reservations_df.loc[reservations_df['name'].str.endswith('/reservations/default')]
if default_row.empty:
    raise ValueError('Could not find default reservation row in reservations_df')

slot_capacity = pd.to_numeric(default_row['slotCapacity'], errors='coerce').fillna(0).iloc[0]
autoscale_current = 0.0
if 'autoscale.currentSlots' in default_row.columns:
    autoscale_current = pd.to_numeric(default_row['autoscale.currentSlots'], errors='coerce').fillna(0).iloc[0]

print(f"Default reservation committed slots: {slot_capacity}")
print(f"Current autoscale slots: {autoscale_current}")

latency_summary['avg_active_slots_pct_capacity'] = latency_summary['avg_active_slots'] / slot_capacity * 100
latency_summary


Default reservation committed slots: 1000
Current autoscale slots: 50


,classification_type,job_count,total_slot_ms,total_run_seconds,avg_active_slots,avg_active_slots_pct_capacity
0,AUTOMATION,4248804,276408165693,2701001,50.044719,5.004472
1,HUB_SERVICE,317534,538361035353,4415326,70.700507,7.070051
2,INTERNAL_USER,8837,140091286165,350526,134.067822,13.406782
3,UNKNOWN,3280,35188663270,147556,89.961729,8.996173


### 10-minute slot usage snapshot
Load `rolling_07d_slot_usage_10min.json`, the 10-minute aggregation produced by `phase3_qos_slot_usage_10min.sql`. This provides per-class slot usage/queue totals used for spike detection.


### Detect spikes via MAD baseline
1. Aggregate total slot-ms per 10-minute bucket.
2. Compute a robust baseline (median + 3 × 1.4826 × MAD).
3. Flag buckets above the threshold and group consecutive buckets into spike events (10-minute cadence).


In [12]:
total_avg_active_slots = latency_summary['avg_active_slots'].sum()
print(f"Aggregate avg active slots across all types: {total_avg_active_slots:.2f}")
print(f"Share of reservation: {total_avg_active_slots / slot_capacity * 100:.2f}%")


Aggregate avg active slots across all types: 344.77
Share of reservation: 34.48%


In [13]:
slot_usage_path = pathlib.Path('rolling_07d_slot_usage_10min.json').resolve()
with open(slot_usage_path) as f:
    slot_usage_raw = json.load(f)
slot_df = pd.json_normalize(slot_usage_raw[0])
slot_numeric_cols = ['job_count', 'total_slot_ms', 'sum_queue_seconds', 'sum_run_seconds', 'sum_total_seconds']
for col in slot_numeric_cols:
    slot_df[col] = pd.to_numeric(slot_df[col], errors='coerce')
slot_df['bucket_ts'] = pd.to_datetime(slot_df['bucket_ts'])
slot_df.head()


,bucket_ts,classification_type,job_count,sum_queue_seconds,sum_run_seconds,sum_total_seconds,total_slot_ms,window_id
0,2025-10-27 23:40:00,AUTOMATION,4,0,7388,7388,1080171219,rolling_07d
1,2025-10-28 00:00:00,AUTOMATION,228,0,5283,5322,430032184,rolling_07d
2,2025-10-28 00:00:00,HUB_SERVICE,64,0,4977,4986,1195609037,rolling_07d
3,2025-10-28 00:10:00,AUTOMATION,421,0,4470,4519,364074833,rolling_07d
4,2025-10-28 00:10:00,HUB_SERVICE,286,0,1510,1549,58103897,rolling_07d


In [14]:
bucket_totals = (
    slot_df.groupby('bucket_ts')['total_slot_ms']
    .sum()
    .reset_index()
    .sort_values('bucket_ts')
)
median_slots = bucket_totals['total_slot_ms'].median()
mad_slots = (bucket_totals['total_slot_ms'] - median_slots).abs().median()
mad_scaled = 1.4826 * mad_slots
threshold = median_slots + 3 * mad_scaled
bucket_totals['is_spike'] = bucket_totals['total_slot_ms'] > threshold
threshold, bucket_totals.head()


(np.float64(2285551523.2398),
             bucket_ts  total_slot_ms  is_spike
 0 2025-10-27 23:40:00     1080171219     False
 1 2025-10-28 00:00:00     1625641221     False
 2 2025-10-28 00:10:00      424328543     False
 3 2025-10-28 00:20:00     1498216971     False
 4 2025-10-28 00:30:00     2539896675      True)

In [15]:
interval = pd.Timedelta(minutes=10)
spike_groups = []
current_group = 0
prev_ts = None
for ts, is_spike in zip(bucket_totals['bucket_ts'], bucket_totals['is_spike']):
    if not is_spike:
        spike_groups.append(pd.NA)
        prev_ts = None
        continue
    if prev_ts is None or ts - prev_ts > interval:
        current_group += 1
    spike_groups.append(current_group)
    prev_ts = ts
bucket_totals['spike_id'] = spike_groups
bucket_totals.head(10)


,bucket_ts,total_slot_ms,is_spike,spike_id
0,2025-10-27 23:40:00,1080171219,False,<NA>
1,2025-10-28 00:00:00,1625641221,False,<NA>
2,2025-10-28 00:10:00,424328543,False,<NA>
3,2025-10-28 00:20:00,1498216971,False,<NA>
4,2025-10-28 00:30:00,2539896675,True,1
5,2025-10-28 00:40:00,1862950089,False,<NA>
6,2025-10-28 00:50:00,316717298,False,<NA>
7,2025-10-28 01:00:00,2095618952,False,<NA>
8,2025-10-28 01:10:00,625380370,False,<NA>
9,2025-10-28 01:20:00,536818285,False,<NA>


In [16]:
spike_buckets = bucket_totals.dropna(subset=['spike_id']).copy()
slot_spike = slot_df.merge(spike_buckets[['bucket_ts', 'spike_id']], on='bucket_ts', how='inner')
slot_spike.head()


,bucket_ts,classification_type,job_count,sum_queue_seconds,sum_run_seconds,sum_total_seconds,total_slot_ms,window_id,spike_id
0,2025-10-28 00:30:00,AUTOMATION,526,0,5057,5142,492606799,rolling_07d,1
1,2025-10-28 00:30:00,HUB_SERVICE,487,1,1442,1490,34987858,rolling_07d,1
2,2025-10-28 00:30:00,UNKNOWN,2,0,13934,13934,2012302018,rolling_07d,1


In [17]:
spike_events = (
    slot_spike.groupby('spike_id')
    .agg(
        start_ts=('bucket_ts', 'min'),
        end_ts=('bucket_ts', 'max'),
        duration_minutes=('bucket_ts', lambda s: (len(s) * 10)),
        total_slot_ms=('total_slot_ms', 'sum'),
        max_slot_ms=('total_slot_ms', 'max'),
        avg_queue_seconds=('sum_queue_seconds', lambda s: s.sum() / max(len(s), 1)),
        classifications=('classification_type', lambda s: s.nunique())
    )
    .reset_index()
)
spike_events['end_ts'] = spike_events['end_ts'] + interval
spike_events


,spike_id,start_ts,end_ts,duration_minutes,total_slot_ms,max_slot_ms,avg_queue_seconds,classifications
0,1,2025-10-28 00:30:00,2025-10-28 00:40:00,30,2539896675,2012302018,0.333333,3


In [18]:
mix_series = slot_spike.groupby(['spike_id', 'classification_type'])['total_slot_ms'].sum()
spike_mix = (mix_series / mix_series.groupby(level=0).transform('sum')).reset_index(name='slot_share')
spike_mix.head()


,spike_id,classification_type,slot_share
0,1,AUTOMATION,0.193948
1,1,HUB_SERVICE,0.013775
2,1,UNKNOWN,0.792277


In [19]:
total_slot_ms_all = slot_df['total_slot_ms'].sum()
spike_events['slot_hours'] = spike_events['total_slot_ms'] / (1000 * 60 * 60)
spike_summary = {
    'spike_count': len(spike_events),
    'spike_days': spike_events['start_ts'].dt.floor('D').nunique(),
    'slot_hours_in_spikes': spike_events['slot_hours'].sum(),
    'share_slot_ms_spikes': spike_events['total_slot_ms'].sum() / total_slot_ms_all if total_slot_ms_all else 0,
    'median_duration_minutes': spike_events['duration_minutes'].median() if len(spike_events) > 0 else 0
}
spike_summary


{'spike_count': 1,
 'spike_days': 1,
 'slot_hours_in_spikes': np.float64(705.5268541666667),
 'share_slot_ms_spikes': np.float64(0.07949126496907455),
 'median_duration_minutes': np.float64(30.0)}

In [20]:
spike_events.to_csv('../rolling_07d_spike_events.csv', index=False)
spike_mix.to_csv('../rolling_07d_spike_mix.csv', index=False)
spike_summary


{'spike_count': 1,
 'spike_days': 1,
 'slot_hours_in_spikes': np.float64(705.5268541666667),
 'share_slot_ms_spikes': np.float64(0.07949126496907455),
 'median_duration_minutes': np.float64(30.0)}

### Rolling-7d Spike Snapshot
- **Spikes detected:** see summary cell below for counts and duration stats.
- **Outputs:**
  - `rolling_07d_spike_events.csv` – spike intervals with slot totals, max load, queue averages.
  - `rolling_07d_spike_mix.csv` – classification share per spike event.
- Re-run the upstream SQL with alternative `window_ids` (e.g., `peak_fy22`, `baseline_fy22`, etc.) and drop the resulting JSON/CSVs in this folder to iterate across historical windows.



In [21]:
print("Spike summary:")
for k, v in spike_summary.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.2f}")
    else:
        print(f"  {k}: {v}")

print("\nTop 5 spike mix rows:")
print(spike_mix.sort_values(['spike_id', 'slot_share'], ascending=[True, False]).head())


Spike summary:
  spike_count: 1
  spike_days: 1
  slot_hours_in_spikes: 705.53
  share_slot_ms_spikes: 0.08
  median_duration_minutes: 30.00

Top 5 spike mix rows:
   spike_id classification_type  slot_share
2         1             UNKNOWN    0.792277
0         1          AUTOMATION    0.193948
1         1         HUB_SERVICE    0.013775


## Notes
- All metrics above are scoped to the rolling 7-day window (easy to expand by editing the `DECLARE window_ids` clause in `phase3_qos_latency_metrics.sql`).
- Slot percentiles are expressed in slot-hours per job; compare to reservation capacity to identify outliers quickly.
- `avg_active_slots` approximates the sustained slot concurrency (slot-ms / runtime). Use it alongside reservation totals (`slot_capacity`, `autoscaleCurrentSlots`) to spot headroom or gaps.
- Next steps: repeat for peak windows, build time-series charts for queue seconds, and integrate reservation assignment events to flag bursts that exceeded the 1000-slot baseline.


In [22]:
slot_percentiles_hours = slot_percentiles / (1000 * 60 * 60)
slot_percentiles_hours


p50            p90  \
classification_type event_name                                               
AUTOMATION          extract_job_completed          8.638612       8.638612   
                    load_job_completed          1437.978812    1437.978812   
                    query_job_completed        75333.428601   75333.428601   
HUB_SERVICE         load_job_completed           429.696384     429.696384   
                    query_job_completed       149115.035658  149115.035658   
INTERNAL_USER       extract_job_completed          0.000189       0.000189   
                    load_job_completed             0.306506       0.306506   
                    query_job_completed        38913.939461   38913.939461   
UNKNOWN             extract_job_completed          0.746337       0.746337   
                    load_job_completed            12.701538      12.701538   
                    query_job_completed         9761.180811    9761.180811   
                    table_copy_job_completed       0.000000       0.000000   

                                                        p99  
classification_type event_name                               
AUTOMATION          extract_job_completed          8.638612  
                    load_job_completed          1437.978812  
                    query_job_completed        75333.428601  
HUB_SERVICE         load_job_completed           429.696384  
                    query_job_completed       149115.035658  
INTERNAL_USER       extract_job_completed          0.000189  
                    load_job_completed             0.306506  
                    query_job_completed        38913.939461  
UNKNOWN             extract_job_completed          0.746337  
                    load_job_completed            12.701538  
                    query_job_completed         9761.180811  
                    table_copy_job_completed       0.000000